In [1]:
    import jinja2, IPython, toolz.curried as toolz, base64, vdom

In [116]:
    mimes = toolz.pipe("""
        50 text/html
        60 text/markdown
        70 text/latex
        80 image/svg+xml
        90 image/bmp image/png image/jpeg image/gif'
        110 text/javascript application/javascript 
        120 text/plain
        """, str.strip, str.splitlines, toolz.map(toolz.compose(toolz.juxt(toolz.compose(int, next), list), iter, str.split)), list)

In [117]:
    image_types = list(map('image/'.__add__, ('bmp', 'jpeg', 'gif', 'png')))

In [5]:
  def ipython_formatter(object):
        try:
            from .macros.utils import format_images
        except:
            from macros.utils import format_images
        format = IPython.get_ipython().display_formatter.format
        bundle, metadata = format(object)
        for type in toolz.pipe(mimes, toolz.map(toolz.second), toolz.concat):
            if type == 'text/plain': return object
            if type in bundle: 
                object = bundle[type]
                if type.startswith('image'):
                    bundle.update(format(format_images(type, bundle))[0])
                return bundle[type]

In [6]:
    environment = jinja2.Environment(enable_async=False, finalize=ipython_formatter)
    environment.globals.update(vars(__import__('builtins')))